In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [7]:
from __future__ import print_function

In [8]:
import os
import glob
import time
import shutil

In [34]:
import chardet

In [68]:
def get_file_encode(fname):
    with open(fname, 'rb') as f:
        enc = chardet.detect(f.read())['encoding']
    return enc

In [112]:
def get_df(fname, years = None):
    
    try:
        df = pd.read_csv(fname)
    except:
        enc = get_file_encode(fname)
        df = pd.read_csv(fname, encoding = enc)
    return df

In [125]:
from collections import OrderedDict

In [9]:
os.getcwd()

'/home/samsay/Documents/Udacity/DSND/DSND-2/StackOverflow'

In [10]:
os.listdir()


['.ipynb_checkpoints',
 'data_all_year',
 '.git',
 '.gitattributes',
 'Stackoverflow.ipynb']

In [16]:
os.listdir('data_all_year')

['2013 Stack Overflow Survey Responses.csv',
 '2017 survey_results_schema_2017.csv',
 '2015 Stack Overflow Developer Survey Responses.csv',
 '2016 Stack Overflow Survey Responses.csv',
 '2014 Stack Overflow Survey Responses.csv',
 '2019 survey_results_public.csv',
 '2012 Stack Overflow Survey Results.csv',
 '2019 survey_results_schema.csv',
 '2017 survey_results_public_2017.csv',
 '2011 Stack Overflow Survey Results.csv',
 '2018 survey_results_schema_2018.csv',
 '2018 survey_results_public_2018.csv']

In [106]:
schema_fname = './data_all_year/2015 Stack Overflow Developer Survey Responses.csv'
#data_fname = './data_all_year/2019 survey_results_public.csv'

In [121]:
df = get_df(schema_fname)
df.head()

/home/samsay/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (5,108,121,196,197,198) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,Country,Age,Gender,Tabs or Spaces,Years IT / Programming Experience,Occupation,Desktop Operating System,Desktop Operating System: write-in,Current Lang & Tech: Android,Current Lang & Tech: Arduino,...,Why use Stack Overflow: I don't use Stack Overflow,How often are Stack Overflow's answers helpful,Why answer: Help a programmer in need,Why answer: Help future programmers,Why answer: Demonstrate expertise,Why answer: Self promotion,Why answer: Sense of responsibility to developers,Why answer: No idea,Why answer: I don't answer and I don't want to,Why answer: I don't answer but I want to
0,Croatia,25-29,Male,Tabs,2 - 5 years,Back-end web developer,Ubuntu,NaN,NaN,NaN,...,NaN,Usually,NaN,NaN,It feels good to demonstrate my expertise.,Demonstrating my expertise will benefit me,I feel a sense of responsibility to the develo...,NaN,NaN,NaN
1,France,20-24,Male,Spaces,1 - 2 years,Back-end web developer,Windows 7,NaN,NaN,NaN,...,NaN,Usually,NaN,My answer will help lots of people who have th...,It feels good to demonstrate my expertise.,NaN,NaN,NaN,NaN,NaN
2,India,20-24,Male,Tabs,1 - 2 years,Back-end web developer,Windows 7,NaN,NaN,NaN,...,NaN,Rarely,NaN,NaN,NaN,Demonstrating my expertise will benefit me,NaN,NaN,NaN,NaN
3,Latvia,25-29,Male,It depends,6 - 10 years,Back-end web developer,Ubuntu,NaN,NaN,NaN,...,NaN,Usually,It feels good to help a programmer in need,My answer will help lots of people who have th...,It feels good to demonstrate my expertise.,Demonstrating my expertise will benefit me,I feel a sense of responsibility to the develo...,NaN,NaN,NaN
4,Norway,30-34,Male,Tabs,2 - 5 years,Back-end web developer,Windows 8,NaN,NaN,NaN,...,NaN,Usually,It feels good to help a programmer in need,NaN,NaN,Demonstrating my expertise will benefit me,NaN,NaN,NaN,"I don't answer much (or at all), but I want to..."


In [212]:
dict_all_df = {}
years = ['2016', '2017', '2018']
for fname in glob.glob('./data_all_year/*.csv'):
    if 'schema' in fname:
        continue
    is_read = True
    file_year = fname.split('/')[-1].split(' ')[0]
    #print(file_year)
    if years is not None:
        if file_year not in years:
            continue
    #print(fname)
    df = get_df(fname)
    dict_all_df[fname] = df
    print(fname, type(dict_all_df[fname]))

./data_all_year/2016 Stack Overflow Survey Responses.csv <class 'pandas.core.frame.DataFrame'>
./data_all_year/2017 survey_results_public_2017.csv <class 'pandas.core.frame.DataFrame'>


/home/samsay/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


./data_all_year/2018 survey_results_public_2018.csv <class 'pandas.core.frame.DataFrame'>


In [213]:
def remove_unnamed_cols(dict_all_df, sorted_keys = True):
    # removing all columns that are unnamed
    new_dict_df = {}
    for fname, df in dict_all_df.items():
        #print(fname)
        #print(df)
        cols = df.columns
        
        new_col = []
        for col_ in cols:
            if 'Unnamed' in col_:
                continue
            #print(col_)
            new_col.append(col_)
        new_dict_df[fname.split('/')[-1]] = df[new_col]
        #break
        print(fname, len(cols), len(new_col))
    if sorted_keys == True:
        dict_all_df = OrderedDict(sorted(new_dict_df.items()))
    new_dict_df = None
    return dict_all_df

In [214]:
dict_all_df = remove_unnamed_cols(dict_all_df)

./data_all_year/2016 Stack Overflow Survey Responses.csv 66 65
./data_all_year/2017 survey_results_public_2017.csv 154 154
./data_all_year/2018 survey_results_public_2018.csv 129 129


In [215]:
print(*dict_all_df, sep = '\n')

2016 Stack Overflow Survey Responses.csv
2017 survey_results_public_2017.csv
2018 survey_results_public_2018.csv


In [216]:
key_ = list(dict_all_df.keys())[0]
df_ = dict_all_df[key_]
df_.head()

,collector,country,un_subregion,so_region,age_range,age_midpoint,gender,self_identification,occupation,occupation_group,...,important_sameend,important_newtech,important_buildnew,important_buildexisting,important_promotion,important_companymission,important_wfh,important_ownoffice,developer_challenges,why_stack_overflow
0,Facebook,Afghanistan,Southern Asia,Central Asia,20-24,22.0,Male,Programmer,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Facebook,Afghanistan,Southern Asia,Central Asia,30-34,32.0,Male,Developer; Engineer; Programmer,Mobile developer - iOS,"Mobile Dev (Android, iOS, WP & Multi-Platform)",...,This is somewhat important,This is very important,This is very important,I don't care about this,This is somewhat important,This is somewhat important,I don't care about this,I don't care about this,Unrealistic expectations; Interacting with non...,To get help for my job; Because I can't do my ...
2,Facebook,Afghanistan,Southern Asia,Central Asia,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Facebook,Afghanistan,Southern Asia,Central Asia,NaN,NaN,Female,Engineer,DevOps,DevOps,...,I don't care about this,This is somewhat important,This is very important,This is very important,I don't care about this,I don't care about this,This is very important,This is somewhat important,Corporate policies; Interacting with non-techn...,To get help for my job; Beacause I love to learn
4,Facebook,Afghanistan,Southern Asia,Central Asia,> 60,65.0,Prefer not to disclose,Developer; Engineer; Programmer; Sr. Developer...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:
df_.isnull().mean() < 0.75

collector                           True
country                             True
un_subregion                        True
so_region                           True
age_range                           True
age_midpoint                        True
gender                              True
self_identification                 True
occupation                          True
occupation_group                    True
experience_range                    True
experience_midpoint                 True
salary_range                        True
salary_midpoint                     True
big_mac_index                       True
tech_do                             True
tech_want                           True
aliens                              True
programming_ability                 True
employment_status                   True
industry                            True
company_size_range                  True
team_size_range                     True
women_on_team                       True
remote          

In [218]:
def remove_null_columns_by_perc(df_, perc = 0.50):
    """
        Function to remove columns that has null values more than 50%
        Keep only columns thtat has less than 50% null
    """
    prev_len = df_.shape[1]
    df_ = df_.loc[:, df_.isnull().mean() <= perc]
    curr_len = df_.shape[1]
    print('Removed: ', prev_len - curr_len, 'Columns')
    return df_

In [219]:
for k, df_ in dict_all_df.items():
    prev_shape = df_.shape
    df_ = remove_null_columns_by_perc(df_, perc = 0.25)
    curr_shape = df_.shape
    print(k, 'Prev: ', prev_shape, ' curr: ', curr_shape)
    print(df_.isnull().mean())
    print()
    

Removed:  12 Columns
2016 Stack Overflow Survey Responses.csv Prev:  (56030, 65)  curr:  (56030, 53)
collector                   0.000000
country                     0.008959
un_subregion                0.012797
so_region                   0.011422
age_range                   0.005408
age_midpoint                0.012386
gender                      0.007924
self_identification         0.032625
occupation                  0.116206
occupation_group            0.162342
experience_range            0.116188
experience_midpoint         0.116188
salary_range                0.176852
big_mac_index               0.128592
tech_do                     0.125022
tech_want                   0.144173
aliens                      0.122309
programming_ability         0.161485
employment_status           0.115188
dev_environment             0.184133
commit_frequency            0.168338
hobby                       0.167000
dogs_vs_cats                0.192593
desktop_os                  0.170962
unit_testin

In [220]:
df_.head()

,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,DevType,YearsCoding,...,VersionControl,AdBlocker,AdsAgreeDisagree1,AdsAgreeDisagree2,AdsAgreeDisagree3,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowJobs,StackOverflowConsiderMember
0,1,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,Full-stack developer,3-5 years,...,Git,Yes,Strongly agree,Strongly agree,Strongly agree,10 (Very Likely),Multiple times per day,Yes,"No, I knew that Stack Overflow had a jobs boar...",Yes
1,3,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...",Database administrator;DevOps specialist;Full-...,30 or more years,...,Git;Subversion,Yes,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,10 (Very Likely),A few times per month or weekly,Yes,Yes,Yes
2,4,Yes,Yes,United States,No,Employed full-time,Associate degree,"Computer science, computer engineering, or sof...",Engineering manager;Full-stack developer,24-26 years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,No,No,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",Full-stack developer,18-20 years,...,Git,Yes,Neither agree nor disagree,Somewhat agree,Somewhat agree,10 (Very Likely),A few times per week,Yes,Yes,Yes
4,7,Yes,No,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",Data or business analyst;Desktop or enterprise...,6-8 years,...,Zip file back-ups,No,Somewhat agree,Somewhat agree,Somewhat disagree,10 (Very Likely),Daily or almost daily,Yes,"No, I knew that Stack Overflow had a jobs boar...",Yes


In [ ]:
def 